In [42]:
import unicodedata

import pandas as pd


def remove_accents(input_str):
    # Normalize the string to decompose accents
    nfkd_form = unicodedata.normalize("NFKD", input_str)
    # Keep only characters that are not combining marks
    return "".join(c for c in nfkd_form if not unicodedata.combining(c))


In [ ]:
def create_codesystem_mcdts_fsh(row, file):
    # print(row)
    if pd.isna(row["cod_sns"]):
        print(row["Nomenclatura"])
    else:
        code = row["cod_sns"]
        code_conv = row["cod_conv"]
        name = row["Nomenclatura"]
        preco = row["preco"]
        taxa = row["taxa"]
        if " " not in str(code):
            file.write(
                "* #"
                + str(code)
                + '\n    "'
                + str(name).replace('"', '\\"')
                + '"\n    "'
                + str(name).replace('"', '\\"')
                + '"\n'
            )
        # print(preco)
        if not pd.isna(preco):
            try:
                file.write(
                    "  * ^property[+].code = #preco\n  * ^property[=].valueDecimal = "
                    + str(round(preco, 2))
                    + "\n  * ^property[+].code = #taxa\n  * ^property[=].valueDecimal = "
                    + str(round(taxa, 2))
                    + "\n"
                )
            except:
                print("ERRROR", preco)


def create_concept_map_mcdts_fsh(row, file):
    # print(row)
    if pd.isna(row["cod_sns"]):
        print(row["Nomenclatura"])
    else:
        code = row["cod_sns"]
        code_conv = row["cod_conv"]

        # print(preco)
        if not pd.isna(code) and not pd.isna(code_conv):
            try:
                file.write(
                    "* group.element[+].code = #"
                    + str(code)
                    + "\n"
                    + "* group.element[=].target.code = #"
                    + str(code_conv)
                    + "\n* group.element[=].target.relationship = #equivalent\n"
                )
            except:
                pass


def process_all_files(LIST, source):
    for area in LIST:
        data = pd.read_excel(
            source,
            sheet_name=area,
        )
        data.columns = ["cod_sns", "cod_conv", "Nomenclatura", "preco", "taxa"]
        name_code = remove_accents(area.replace(" ", "-").lower())
        name_id = remove_accents(area.replace(" ", "").replace("-", "_"))
        with open("../input/fsh/codesystems/mcdts-" + name_code + ".fsh", "w") as file:
            file.write(
                """// Define a local code system
CodeSystem: MCDTS"""
                + name_id
                + """CS
Id:         mcdts-"""
                + name_code
                + """-cs
Title:     "CodeSystem MCDTS """
                + name_code
                + """"
Description: "CodeSystem """
                + name_code
                + """"

* ^url =  "https://terminologias.hl7.pt/fhir/CodeSystem/mcdts-"""
                + name_code
                + """-cs"
* ^version = "5.9.2024"
* ^author.name = "ACSS"


* ^property[+].code = #preco
* ^property[=].description = "Preço do MCDT"
* ^property[=].type = #decimal
* ^property[+].code = #taxa
* ^property[=].description = "Taxa Moderadora"
* ^property[=].type = #decimal

"""
            )
            row_sums = data.iloc[3:, :].apply(
                create_codesystem_mcdts_fsh, axis=1, file=file
            )  # Axis 1 applies the function row-wise

        with open(
            "../input/fsh/conceptmaps/mcdts-" + name_code + "-sns-conv.fsh", "w"
        ) as file:
            file.write(
                """
Instance: mcdts-"""
                + name_code
                + """-sns-conv
InstanceOf: ConceptMap
Usage: #definition

* version = "5.9.2024"
* name = "MCDTS-"""
                + name_code
                + """-SNS-CONV"
* title = "Mapeamento de códigos SNS para códigos convencionados para a área """
                + name_code
                + """ "

* status = #draft
* experimental = true
* date = "2024-12-11"
* publisher = "ACSS / HL7 Portugal"

* group.source = "https://terminologias.hl7.pt/fhir/CodeSystem/mcdts-"""
                + name_code
                + """-cs"
* group.target = "https://www.acss.pt"
                       
"""
            )
            row_sums = data.iloc[3:, :].apply(
                create_concept_map_mcdts_fsh, axis=1, file=file
            )  # Axis 1 applies the function row-wise

In [44]:
LIST = [
    "A-Análises",
    "B-Anatomia",
    "C-Cardiologia",
    "D-M Nuclear",
    "E-EEG",
    "M-Radiologia",
    "F-Gastrenterologia",
    "G-MFR",
    "H-ORL",
    "J-Urologia",
    "I-Pneumologia-Imunoalergologia",
    "L-Neurofisiologia",
]
source = "/Users/joaoalmeida/Downloads/Tabela-MCDT-Convencionados-2024_05.09.2024.xlsx"
process_all_files(LIST, source)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

BIOQUÍMICA
ERRROR a)
ERRROR a)
ERRROR a)
Equilíbrio ácido-base (pH, pCO2, pO2, SatO2, CO2, ...), s (ver código 21789 - Gasimetria…)
ERRROR a)
ERRROR a)
Morfina (ver código 21775 - Drogas de abuso, pesquisa, cada, s/u)
ERRROR a)
ERRROR a)
ERRROR a)
nan
HEMATOLOGIA
ERRROR a)
nan
HEMOSTASE
ERRROR a)
nan
IMUNOLOGIA
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
CITOMETRIA DE FLUXO
O estudo por citometria de fluxo é um estudo de carácter interpretativo, que pode exigir uma abordagem sequencial. Assim, em alguns casos, foi contemplada a possibilidade de dois tipos de estudo que podem ser efetuados sequencialmente na mesma amostra em caso de necessidade: um primeiro estudo, designado por estudo inicial e um segundo, designado por estudo complementar.
Anticorpos, pesquisa em células e em soro
Doenças linfoproliferativas, imunofenotipagem
Imunodeficiências e doenças autoimunes, caracterização de distúrbios da imunidade
nan
MICROBIOLOGIA
Serologia
Os códigos que não explic

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

nan
ELECTROENCEFALOGRAFIA
nan
nan
nan
nan
ELECTROENCEFALOGRAFIA
nan
nan
nan
TABELA DE RADIOLOGIA
RADIOLOGIA CONVENCIONAL
CABEÇA E PESCOÇO
Exames especiais de cabeça e pescoço
nan
COLUNA VERTEBRAL E BACIA
nan
TÓRAX
nan
MAMA
ERRROR a)
nan
ABDÓMEN E TRACTO DIGESTIVO
Abdómen e Pélvis
Tracto Digestivo
ERRROR a)
nan
APARELHO GENITO-URINÁRIO
ERRROR a)
nan
ESQUELETO APENDICULAR
Membros superiores
Membros inferiores
Exames especiais membros 
nan
TOMOGRAFIAS CLÁSSICAS (CONVENCIONAIS)
nan
ECOGRAFIA
Cabeça e pescoço
nan
nan
Mama
Abdómen e pélvis
ERRROR a)
ERRROR a)
ERRROR a)
Nota explicativa 1: Em gravidez múltipla, nas ecografias do 2ºe 3º trimestres, por cada feto é registado um exame.
Nota explicativa 2: Para os códigos 17190, 17195 e 17197 é obrigatório que o médico que executa o exame detenha a competência em ecografia obstétrica diferenciada.
Sistema músculo-esquelético
ERRROR a)
Estudos por Doppler (duplex ou triplex)
Ecocardiograma transtorácico bidimensional (ver tabela de Cardiologia, có

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

Actos complementares de diagnóstico
Primeira Consulta em Medicina Física e de Reabilitação
Consulta Subsequente em Medicina Física e de Reabilitação
Consulta de Avaliação Final em Medicina Física e de Reabilitação
nan
Estudos especificos
Provas funcionais respiratórias 
Espirometria, incluindo gráficos, curva débito volume, capacidade vital, volume expiratório máximo no 1º segundo, débito expiratório máximo e médio e ventilação voluntária máxima por minuto (ver tabela de Pneumologia, código 1504.5)
Capacidade residual funcional ou volume residual (hélio/azoto) (ver tabela de Pneumologia, código 1505.3)
Estudo da resistência das vias aéreas e volume de gás intra-toracico por pletismografia corporal (ver tabela de Pneumologia, código 1506.1)
Prova de broncodilatação (acresce à prova basal) (ver tabela de Pneumologia, código 1507.0)
Prova de broncoconstrição com água destilada/soluto hipertónico (acresce à prova basal) (ver tabela de Pneumologia, código 1508.8)
Prova de broncoconstrição d

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d